# SA with Shapely Validation

Using Shapely for strict overlap validation to ensure valid submissions.
Start from local_search baseline (87.36) and try to improve.

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import time
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25
scale_factor = Decimal('1e15')
SCALE = float(scale_factor)

print("Libraries loaded")

Libraries loaded


In [2]:
# Tree template
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125], np.float64)
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5], np.float64)

def get_tree_vertices(cx, cy, deg):
    """Get rotated and translated tree vertices."""
    angle_rad = deg * math.pi / 180.0
    c = math.cos(angle_rad)
    s = math.sin(angle_rad)
    vx = c * TX - s * TY + cx
    vy = s * TX + c * TY + cy
    return vx, vy

def create_polygon(cx, cy, deg):
    """Create Shapely polygon for a tree."""
    vx, vy = get_tree_vertices(cx, cy, deg)
    return Polygon(list(zip(vx, vy)))

def get_bbox_side(xs, ys, degs):
    """Calculate bounding box side length."""
    all_vx, all_vy = [], []
    for i in range(len(xs)):
        vx, vy = get_tree_vertices(xs[i], ys[i], degs[i])
        all_vx.extend(vx)
        all_vy.extend(vy)
    return max(max(all_vx) - min(all_vx), max(all_vy) - min(all_vy))

def score_group(xs, ys, degs):
    """Calculate score (side^2 / n)."""
    side = get_bbox_side(xs, ys, degs)
    return side * side / len(xs)

print("Helper functions defined")

Helper functions defined


In [3]:
def has_overlap_shapely(xs, ys, degs, area_threshold=1e-15):
    """Check for overlaps using Shapely with area threshold."""
    n = len(xs)
    if n <= 1:
        return False
    
    polygons = [create_polygon(xs[i], ys[i], degs[i]) for i in range(n)]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx <= i:
                continue
            if poly.intersects(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                if intersection.area > area_threshold:
                    return True
    return False

def check_single_overlap_shapely(xs, ys, degs, idx, area_threshold=1e-15):
    """Check if tree at idx overlaps with any other."""
    n = len(xs)
    if n <= 1:
        return False
    
    poly_i = create_polygon(xs[idx], ys[idx], degs[idx])
    
    for j in range(n):
        if j == idx:
            continue
        poly_j = create_polygon(xs[j], ys[j], degs[j])
        if poly_i.intersects(poly_j):
            intersection = poly_i.intersection(poly_j)
            if intersection.area > area_threshold:
                return True
    return False

print("Shapely overlap functions defined (with area threshold)")

Shapely overlap functions defined (with area threshold)


In [4]:
# Load the best existing submission (local_search: 87.36)
df = pd.read_csv('/home/code/submission_candidates/candidate_002.csv')
print(f"Loaded candidate_002.csv: {len(df)} rows")

# Parse the data
def strip_s(val):
    return float(str(val).replace('s', ''))

df['x_val'] = df['x'].apply(strip_s)
df['y_val'] = df['y'].apply(strip_s)
df['deg_val'] = df['deg'].apply(strip_s)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

# Calculate current score
current_scores = {}
for n in range(1, 201):
    group = df[df['n'] == n]
    xs = group['x_val'].values
    ys = group['y_val'].values
    degs = group['deg_val'].values
    current_scores[n] = score_group(xs, ys, degs)

total_score = sum(current_scores[n] for n in range(1, 201))
print(f"Current total score: {total_score:.6f}")

Loaded candidate_002.csv: 20100 rows


Current total score: 87.364112


In [5]:
def find_best_grid_trees(n):
    """Find the best grid-based placement for n trees."""
    best_score = float('inf')
    best_xs, best_ys, best_degs = None, None, None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            
            xs_list, ys_list, degs_list = [], [], []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0.0 if r % 2 == 0 else 180.0
                x_offset = 0.0 if r % 2 == 0 else 0.35
                
                if r % 2 == 0:
                    y = float(r // 2) * 1.0
                else:
                    y = 0.8 + float((r - 1) // 2) * 1.0
                
                for i in range(m):
                    xs_list.append(0.7 * i + x_offset)
                    ys_list.append(y)
                    degs_list.append(angle)
                
                r += 1
            
            if len(xs_list) != n:
                continue
            
            xs = np.array(xs_list, np.float64)
            ys = np.array(ys_list, np.float64)
            degs = np.array(degs_list, np.float64)
            
            score = score_group(xs, ys, degs)
            if score < best_score:
                best_score = score
                best_xs = xs.copy()
                best_ys = ys.copy()
                best_degs = degs.copy()
    
    return best_xs, best_ys, best_degs, best_score

print("Grid function defined")

Grid function defined


In [6]:
def sa_optimize_safe(xs, ys, degs, max_iter=1000, T0=1.0, alpha=0.95, seed=42):
    """
    SA optimizer with Shapely validation for safety.
    """
    random.seed(seed)
    np.random.seed(seed)
    n = len(xs)
    
    if n <= 1:
        return xs.copy(), ys.copy(), degs.copy(), score_group(xs, ys, degs)
    
    cur_xs = xs.copy()
    cur_ys = ys.copy()
    cur_degs = degs.copy()
    
    best_xs = xs.copy()
    best_ys = ys.copy()
    best_degs = degs.copy()
    
    cur_score = score_group(cur_xs, cur_ys, cur_degs)
    best_score = cur_score
    
    T = T0
    move_scale = 0.1
    rot_scale = 30.0
    no_improve = 0
    
    for iteration in range(max_iter):
        scale = T / T0
        
        i = random.randint(0, n - 1)
        move_type = random.randint(0, 3)
        
        orig_x = cur_xs[i]
        orig_y = cur_ys[i]
        orig_deg = cur_degs[i]
        
        cx = cur_xs.mean()
        cy = cur_ys.mean()
        
        if move_type == 0:  # Random translation
            cur_xs[i] += (random.random() - 0.5) * 2 * move_scale * scale
            cur_ys[i] += (random.random() - 0.5) * 2 * move_scale * scale
        elif move_type == 1:  # Centroid move
            dx = (cx - cur_xs[i]) * random.random() * scale * 0.3
            dy = (cy - cur_ys[i]) * random.random() * scale * 0.3
            cur_xs[i] += dx
            cur_ys[i] += dy
        elif move_type == 2:  # Random rotation
            cur_degs[i] += (random.random() - 0.5) * 2 * rot_scale * scale
        else:  # Combined
            cur_xs[i] += (random.random() - 0.5) * 2 * move_scale * scale
            cur_ys[i] += (random.random() - 0.5) * 2 * move_scale * scale
            cur_degs[i] += (random.random() - 0.5) * 2 * rot_scale * scale
        
        # Check for overlap using Shapely
        if check_single_overlap_shapely(cur_xs, cur_ys, cur_degs, i):
            cur_xs[i] = orig_x
            cur_ys[i] = orig_y
            cur_degs[i] = orig_deg
            move_scale *= 0.99
            rot_scale *= 0.99
            continue
        
        new_score = score_group(cur_xs, cur_ys, cur_degs)
        
        delta = new_score - cur_score
        if delta < 0 or random.random() < math.exp(-delta / T):
            cur_score = new_score
            move_scale = min(move_scale * 1.01, 0.1)
            rot_scale = min(rot_scale * 1.01, 30.0)
            
            if cur_score < best_score:
                best_score = cur_score
                best_xs = cur_xs.copy()
                best_ys = cur_ys.copy()
                best_degs = cur_degs.copy()
                no_improve = 0
            else:
                no_improve += 1
        else:
            cur_xs[i] = orig_x
            cur_ys[i] = orig_y
            cur_degs[i] = orig_deg
            move_scale *= 0.99
            rot_scale *= 0.99
            no_improve += 1
        
        T *= alpha
        
        if no_improve > 200:
            T = min(T * 3.0, T0 * 0.7)
            no_improve = 0
    
    return best_xs, best_ys, best_degs, best_score

print("Safe SA optimizer defined")

Safe SA optimizer defined


In [7]:
# Test on N=10
print("Testing SA on N=10...")
xs, ys, degs, grid_score = find_best_grid_trees(10)
print(f"Grid score: {grid_score:.6f}")

opt_xs, opt_ys, opt_degs, sa_score = sa_optimize_safe(xs, ys, degs, max_iter=1000, seed=42)
print(f"SA score: {sa_score:.6f}")
print(f"Has overlap: {has_overlap_shapely(opt_xs, opt_ys, opt_degs)}")

Testing SA on N=10...
Grid score: 0.484000


SA score: 0.484000
Has overlap: False


In [8]:
# Debug: Check grid baseline for overlaps
print("Checking grid baseline for N=10...")
print(f"Grid has overlap: {has_overlap_shapely(xs, ys, degs)}")

# The issue might be that the grid itself has overlaps
# Let's check the tree positions
print(f"\nTree positions:")
for i in range(len(xs)):
    print(f"  Tree {i}: x={xs[i]:.4f}, y={ys[i]:.4f}, deg={degs[i]:.1f}")

Checking grid baseline for N=10...
Grid has overlap: False

Tree positions:
  Tree 0: x=0.0000, y=0.0000, deg=0.0
  Tree 1: x=0.7000, y=0.0000, deg=0.0
  Tree 2: x=1.4000, y=0.0000, deg=0.0
  Tree 3: x=0.3500, y=0.8000, deg=180.0
  Tree 4: x=1.0500, y=0.8000, deg=180.0
  Tree 5: x=0.0000, y=1.0000, deg=0.0
  Tree 6: x=0.7000, y=1.0000, deg=0.0
  Tree 7: x=1.4000, y=1.0000, deg=0.0
  Tree 8: x=0.3500, y=1.8000, deg=180.0
  Tree 9: x=1.0500, y=1.8000, deg=180.0


In [9]:
# Debug: Check which pairs overlap
print("Checking all pairs for overlap...")
polygons = [create_polygon(xs[i], ys[i], degs[i]) for i in range(len(xs))]

for i in range(len(polygons)):
    for j in range(i + 1, len(polygons)):
        if polygons[i].intersects(polygons[j]):
            touches = polygons[i].touches(polygons[j])
            intersection = polygons[i].intersection(polygons[j])
            print(f"  Pair ({i}, {j}): intersects=True, touches={touches}, intersection_area={intersection.area:.2e}")

Checking all pairs for overlap...
  Pair (0, 1): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (0, 3): intersects=True, touches=False, intersection_area=7.46e-33
  Pair (0, 5): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (1, 2): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (1, 3): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (1, 4): intersects=True, touches=False, intersection_area=6.16e-33
  Pair (1, 6): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (2, 4): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (2, 7): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (3, 4): intersects=True, touches=False, intersection_area=6.16e-33
  Pair (3, 5): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (3, 6): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (3, 8): intersects=True, touches=True, intersection_area=0.00e+00
  Pair (4, 6):

In [9]:
# Generate optimized solutions for all N
print("Generating solutions for N=1 to 200...")
print("Using SA with Shapely validation")

start_time = time.time()
solutions = {}
scores = {}

for n in tqdm(range(1, 201)):
    if n == 1:
        # Optimal N=1 at 45 degrees
        xs = np.array([0.0])
        ys = np.array([0.0])
        degs = np.array([45.0])
        scores[n] = score_group(xs, ys, degs)
        solutions[n] = (xs, ys, degs)
        continue
    
    # Get grid baseline
    xs, ys, degs, grid_score = find_best_grid_trees(n)
    
    # Determine iterations based on N
    if n <= 20:
        iterations = 2000
    elif n <= 50:
        iterations = 1000
    elif n <= 100:
        iterations = 500
    else:
        iterations = 300
    
    # Run SA
    opt_xs, opt_ys, opt_degs, sa_score = sa_optimize_safe(xs, ys, degs, max_iter=iterations, seed=42+n)
    
    # Validate and use better solution
    if not has_overlap_shapely(opt_xs, opt_ys, opt_degs) and sa_score < grid_score:
        solutions[n] = (opt_xs, opt_ys, opt_degs)
        scores[n] = sa_score
    else:
        solutions[n] = (xs, ys, degs)
        scores[n] = grid_score

elapsed = time.time() - start_time
print(f"\nTotal time: {elapsed/60:.1f} minutes")

Generating solutions for N=1 to 200...
Using SA with Shapely validation


  0%|          | 0/200 [00:00<?, ?it/s]

  1%|          | 2/200 [00:00<00:17, 11.22it/s]

  2%|▏         | 4/200 [00:00<00:40,  4.86it/s]

  2%|▎         | 5/200 [00:01<00:51,  3.82it/s]

...

 99%|█████████▉| 198/200 [02:37<00:02,  1.29s/it]

100%|█████████▉| 199/200 [02:38<00:01,  1.33s/it]

100%|██████████| 200/200 [02:39<00:00,  1.34s/it]

100%|██████████| 200/200 [02:39<00:00,  1.25it/s]


Total time: 2.7 minutes


In [10]:
# Calculate total score
total_score = sum(scores[n] for n in range(1, 201))
print(f"Total score: {total_score:.6f}")
print(f"Target: 68.947559")
print(f"Gap: {total_score - 68.947559:.6f}")

# Compare with previous best
prev_best = 87.364112
print(f"\nPrevious best: {prev_best:.6f}")
print(f"Improvement: {prev_best - total_score:.6f}")

# Score breakdown
print("\nScore breakdown:")
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(scores[n] for n in range(start, end + 1))
    print(f"  N={start:3d}-{end:3d}: {range_score:.4f}")

Total score: 87.832212
Target: 68.947559
Gap: 18.884653

Previous best: 87.364112
Improvement: -0.468100

Score breakdown:
  N=  1- 10: 6.3122
  N= 11- 50: 19.3587
  N= 51-100: 21.4551
  N=101-150: 20.5883
  N=151-200: 20.1178


In [11]:
# Validate all configurations
print("Validating all configurations...")
overlap_issues = []

for n in range(1, 201):
    xs, ys, degs = solutions[n]
    if has_overlap_shapely(xs, ys, degs):
        overlap_issues.append(n)
        print(f"  N={n}: OVERLAP!")

if overlap_issues:
    print(f"\nWARNING: Overlaps in {len(overlap_issues)} groups!")
else:
    print("\n✓ All valid!")

Validating all configurations...



✓ All valid!


In [12]:
# Create submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n in range(1, 201):
    xs, ys, degs = solutions[n]
    for i in range(n):
        rows.append({
            "id": f"{n:03d}_{i}",
            "x": to_str(xs[i]),
            "y": to_str(ys[i]),
            "deg": to_str(degs[i]),
        })

submission = pd.DataFrame(rows)
print(f"Submission shape: {submission.shape}")
print(submission.head(10))

Submission shape: (20100, 4)
      id           x           y          deg
0  001_0        s0.0        s0.0        s45.0
1  002_0  s-0.037886   s0.135107         s0.0
2  002_1   s0.390949   s0.833417  s196.326006
3  003_0        s0.0        s0.0         s0.0
4  003_1   s0.697606  s-0.083624   s-9.675991
5  003_2       s0.35        s0.8       s180.0
6  004_0        s0.0        s0.0         s0.0
7  004_1        s0.7        s0.0         s0.0
8  004_2       s0.35        s0.8       s180.0
9  004_3       s1.05        s0.8       s180.0


In [13]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/003_high_iter_sa/submission.csv', index=False)
print("Saved!")
print(f"\nFinal score: {total_score:.6f}")

Saved!

Final score: 87.832212
